In [ ]:
"""import shutil

for f in os.listdir('/kaggle/working/'):
    shutil.rmtree(f)"""

In [1]:
!wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv

--2023-04-14 16:00:04--  https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.194.128, 173.194.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2258447590 (2.1G) [text/csv]
Saving to: ‘oidv6-train-annotations-bbox.csv’

oidv6-train-annotat 100%[===================>]   2.10G   164MB/s    in 12s     

2023-04-14 16:00:16 (181 MB/s) - ‘oidv6-train-annotations-bbox.csv’ saved [2258447590/2258447590]



In [2]:
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv

--2023-04-14 16:00:32--  https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.128, 142.251.6.128, 108.177.112.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25105048 (24M) [text/csv]
Saving to: ‘validation-annotations-bbox.csv’

validation-annotati 100%[===================>]  23.94M  --.-KB/s    in 0.1s    

2023-04-14 16:00:33 (196 MB/s) - ‘validation-annotations-bbox.csv’ saved [25105048/25105048]



In [3]:
!wget https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv

--2023-04-14 16:00:34--  https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.194.128, 173.194.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77484237 (74M) [text/csv]
Saving to: ‘test-annotations-bbox.csv’

test-annotations-bb 100%[===================>]  73.89M   198MB/s    in 0.4s    

2023-04-14 16:00:34 (198 MB/s) - ‘test-annotations-bbox.csv’ saved [77484237/77484237]



In [4]:
!git clone https://github.com/mohamedamine99/YOLOv8-custom-object-detection

Cloning into 'YOLOv8-custom-object-detection'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 57 (delta 22), reused 38 (delta 10), pack-reused 7
Receiving objects: 100% (57/57), 80.79 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [5]:
# Create a list of annotated images to be downloaded by the downloader script download.py
# the list is a text file in the following format : $SPLIT/$IMAGE_ID
# example : 
# train/f9e0434389a1d4dd
# train/1a007563ebc18664

import os

alpaca_id = '/m/0pcr'

train_bboxes_filename = os.path.join('.', 'oidv6-train-annotations-bbox.csv')
validation_bboxes_filename = os.path.join('.', 'validation-annotations-bbox.csv')
test_bboxes_filename = os.path.join('.', 'test-annotations-bbox.csv')

image_list_file_path = os.path.join('.', 'image_list_file.txt')

image_list_file_list = []
for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in [alpaca_id] and id not in image_list_file_list:
                image_list_file_list.append(id)
                with open(image_list_file_path, 'a') as fw:
                    fw.write('{}/{}\n'.format(['train', 'validation', 'test'][j], id))
            line = f.readline()

        f.close()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [6]:
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

--2023-04-14 16:01:09--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   4.14K  --.-KB/s    in 0s      

2023-04-14 16:01:09 (40.8 MB/s) - ‘downloader.py’ saved [4244/4244]



In [7]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 68.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.113 which is incompatible.


In [8]:
import os
import shutil


DATA_ALL_DIR = os.path.join('.', 'data_all') # all downloaded data selected from the list
DATA_OUT_DIR = os.path.join('.', 'data') # data reorganized in YOLO format
os.makedirs(DATA_ALL_DIR)
os.makedirs(DATA_OUT_DIR)

In [9]:
!python downloader.py ./image_list_file.txt --download_folder=/kaggle/working/data_all --num_processes=5

In [10]:
# Create a dataset in the yolo format with two main directories : images and labels
# each containing train and validation data 

#import os
#import shutil
#

#DATA_ALL_DIR = os.path.join('.', 'data_all') # all downloaded data selected from the list
#DATA_OUT_DIR = os.path.join('.', 'data') # data reorganized in YOLO format

#os.makedirs(DATA_ALL_DIR)
#os.makedirs(DATA_OUT_DIR)

for dir_ in ['images', 'labels']:
    for set_ in ['train', 'validation', 'test']:
        new_dir = os.path.join(DATA_OUT_DIR, dir_, set_)
        if os.path.exists(new_dir):
            shutil.rmtree(new_dir)
        os.makedirs(new_dir)


In [11]:
alpaca_id = '/m/0pcr'

train_bboxes_filename = os.path.join('.', 'oidv6-train-annotations-bbox.csv')
validation_bboxes_filename = os.path.join('.', 'validation-annotations-bbox.csv')
test_bboxes_filename = os.path.join('.', 'test-annotations-bbox.csv')


for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    set_ = ['train', 'validation', 'test'][j]
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in [alpaca_id]:

                if not os.path.exists(os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id))):

                    shutil.copy(os.path.join(DATA_ALL_DIR, '{}.jpg'.format(id)),
                                os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id)))
                    
                with open(os.path.join(DATA_OUT_DIR, 'labels', set_, '{}.txt'.format(id)), 'a') as f_ann:
                    # class_id, xc, yx, w, h
                    x1, x2, y1, y2 = [float(j) for j in [x1, x2, y1, y2]]
                    xc = (x1 + x2) / 2
                    yc = (y1 + y2) / 2
                    w = x2 - x1
                    h = y2 - y1

                    f_ann.write('0 {} {} {} {}\n'.format(xc, yc, w, h))
                    f_ann.close()

            line = f.readline()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [12]:
%pip install ultralytics
import ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 kB 9.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 12.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60
Note: you may need to restart the kernel to use updated packages.


In [13]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.pt")  # build a new model from scratch

# Use the model
results = model.train(data='/kaggle/working/YOLOv8-custom-object-detection/Custom-object-detection-with-YOLOv8/config -kaggle-.yaml', 
                      epochs=150)  # train the model


  0%|          | 0.00/6.23M [00:00<?, ?B/s]

Ultralytics YOLOv8.0.77 🚀 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla T4, 15110MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/YOLOv8-custom-object-detection/Custom-object-detection-with-YOLOv8/config -kaggle-.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, form

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias
train: Scanning /kaggle/working/data/labels/train... 452 images, 0 backgrounds, 0 corrupt: 100%|██████████| 452/452 [00:00<00:00, 1862.08it/s]
train: New cache created: /kaggle/working/data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /kaggle/working/data/labels/validation... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<00:00, 2750.83it/s]
val: New cache created: /kaggle/working/data/labels/validation.cache
Plotting labels to runs/detect/train/labels.jpg... 
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to runs/detect/train
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_los

In [14]:
!zip -r train_results.zip /kaggle/working/runs

  adding: kaggle/working/runs/ (stored 0%)
  adding: kaggle/working/runs/detect/ (stored 0%)
  adding: kaggle/working/runs/detect/train/ (stored 0%)
  adding: kaggle/working/runs/detect/train/PR_curve.png (deflated 27%)
  adding: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 39%)
  adding: kaggle/working/runs/detect/train/val_batch0_pred.jpg (deflated 11%)
  adding: kaggle/working/runs/detect/train/R_curve.png (deflated 17%)
  adding: kaggle/working/runs/detect/train/events.out.tfevents.1681488174.f0a5564cd3ed.23.0 (deflated 71%)
  adding: kaggle/working/runs/detect/train/P_curve.png (deflated 16%)
  adding: kaggle/working/runs/detect/train/labels.jpg (deflated 24%)
  adding: kaggle/working/runs/detect/train/train_batch2.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/train/confusion_matrix.png (deflated 39%)
  adding: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/train/F1_curve.png (deflated 16%)
  addin